In [ ]:
#basic packages
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

#interpolation
from scipy import interpolate 
#model
from statsmodels.tsa.api import VAR
#classifcation metrics
from sklearn.metrics import f1_score, roc_auc_score
#beta diversity
from scipy.spatial import distance
#alpha diveristy metrics
from skbio.diversity.alpha import shannon
#ordination
import skbio
from scipy.spatial import procrustes

sns.set_style("whitegrid")

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
%cd

# 1. READ FEATURE TABLE AND METADATA FILES

In [ ]:
df = pd.read_csv('raw_data/550_male_feces.tsv',
                 sep = '\t',
                 index_col = [0])
metadata = pd.read_csv('raw_data/550_metadata.txt',
                       sep = '\t')

# 3. RAREFY, ASSIGN TAXONOMY AND COLLAPSE TO FAMILY LEVEL
    1. export to biom
    2. import to QIIME2 artifact
    3. rarefy sequnce feature table
    4. assign taxonomy
    5. collapse to family level

In [ ]:
# we need a dataframe with sequences in rows and samples as columns
rarefy_df = nearest_df.T
rarefy_df.index.name = '#OTU ID'
rarefy_df.to_csv('interpolated_data/interpolated_male_feces.tsv', sep = '\t')

In [ ]:
#convert to biom
biom convert \
-i interpolated_data/interpolated_male_feces.tsv \
-o interpolated_data/interpolated_male_feces.biom \
--to-hdf5

#import to qiime artifact
qiime tools import \
--input-path interpolated_data/interpolated_male_feces.biom \
--type 'FeatureTable[Frequency]' \
--input-format BIOMV210Format \
--output-path interpolated_data/interpolated_male_feces.qza 

#rarefy data
qiime feature-table rarefy \
--i-table interpolated_data/interpolated_male_feces.qza \
--p-sampling-depth 16000 \
--o-rarefied-table rarefied_and_interpolated_data/rarefied_interpolated_male_feces.qza 

# ASSIGN TAXONOMY
## import fasta file with sequences to qza
xxxxxx

## filter repsequences by sequences that are male (in our featrue table)
qiime feature-table filter-seqs \
--i-data sequence_data/sequences.qza \
--i-table rarefied_and_interpolated_data/rarefied_interpolated_male_feces.qza \
--o-filtered-data sequence_data/rarefied_interpolated_male_feces_sequences.qza 

#assign taxonomy
qiime feature-classifier classify-sklearn \
  --i-classifier sequence_data/gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads sequence_data/rarefied_interpolated_male_feces_sequences.qza \
  --o-classification taxonomy/rarefied_interpolated_male_feces_taxonomy.qza

#collapse
qiime taxa collapse \
--i-table rarefied_and_interpolated_data/rarefied_interpolated_male_feces.qza \
--i-taxonomy taxonomy/rarefied_interpolated_male_feces_taxonomy.qza \
--p-level 5 \
--o-collapsed-table taxonomy/rarefied_interpolated_male_feces_family.qza

#export to biom 
qiime tools export \
--input-path taxonomy/rarefied_interpolated_male_feces_family.qza \
--output-path taxonomy/output

#export to tsv
biom convert \
-i taxonomy/output/feature-table.biom \
-o taxonomy/rarefied_interpolated_male_feces_family.tsv \
--to-tsv

### 3.1 read feature table assigned to taxonomy level

In [ ]:
family_df = pd.read_csv('taxonomy/rarefied_interpolated_male_feces_family.tsv',
                        sep = '\t', skiprows = [0], index_col = [0])

#remove bacteria that have abundance below 0.0001 of the highest value
mean_family = pd.DataFrame(family_df.T.describe().loc['mean']).sort_values(by = ['mean'])
keep_families = mean_family[mean_family['mean'] > 0.848].index
family_df = family_df.loc[keep_families]

In [ ]:
#make dataframe to interpolate
interpolate_family = family_df.T
interpolate_family.index = interpolate_family.index.astype(int)
#define missing datapoints
missing_timepoints = list(set([i for i in range(0, 419)]) - set(interpolate_family.index))
#add these points into family dataframe
family_with_missing_tpoints = interpolate_family.reindex(interpolate_family.index.union(missing_timepoints))

#interpolate using nearest interpolation method
interpolated_timepoints = []
for col in family_with_missing_tpoints.columns:
    y = nearest_interp(col, family_with_missing_tpoints)
    interpolated_timepoints.append(y)
    
nearest_family_df = pd.concat(interpolated_timepoints, axis=1)

#slice dataframe, other parts are of low quality
nearest_family_df = nearest_family_df.loc[112:391].T
nearest_family_df.index.name = '#OTU ID'

# 4. NAIVE FORECAST - lag0

In [ ]:
data = nearest_family_df.T
naive_predictions = data.apply(lambda x: x.shift(1), axis = 0)[-40:]
true = data[-40:]

# 5. VAR MODEL

### 5.1. split to train and test

In [ ]:
df_train = data.head(240)
df_test = data.tail(40)

In [ ]:
def var_model(lag_order, n):

    lag_order = lag_order
    nobs = n

    X = df_train.values
    y = df_test.values

    history = [x for x in X]

    #model
    VAR_model = VAR(X)
    VAR_model_fit = VAR_model.fit(lag_order)

    # make first prediction
    predictions = list()
    forecast_input = X[-lag_order:]

    yhat = VAR_model_fit.forecast(y=forecast_input, steps=nobs)

    predictions.append(yhat)
    history.append(y[0])

    for i in range(1, len(y)):

        # predict
        VAR_model = VAR(history)
        VAR_model_fitted = VAR_model.fit(lag_order)

        forecast_idx = i + lag_order - 1
        yhat = VAR_model_fitted.forecast(y=y[:forecast_idx], steps=nobs)
        #yhat = VAR_model_fitted.forecast(y=history[-lag_order:], steps=nobs)
        
        predictions.append(yhat)
        obs = y[i]
        history.append(obs)
        
    return history, predictions, VAR_model_fitted

In [ ]:
def forecast_to_datafarme(history, predictions):
    
    history = pd.DataFrame(history)
    history.columns, history.index = df_train.columns, data.index

    PREDICTIONS = []
    for i in predictions:
        d = pd.DataFrame(i)
        PREDICTIONS.append(d)

    PREDICTIONS = pd.concat(PREDICTIONS)
    PREDICTIONS.columns, PREDICTIONS.index = df_train.columns, df_test.index
    PREDICTIONS[PREDICTIONS < 0] = 0
    
    return history, PREDICTIONS

In [ ]:
HISTORY_1, PREDICTIONS_1 = forecast_to_datafarme(var_model(1, 1)[0], var_model(1, 1)[1])
HISTORY_2, PREDICTIONS_2 = forecast_to_datafarme(var_model(2, 1)[0], var_model(2, 1)[1])
HISTORY_3, PREDICTIONS_3 = forecast_to_datafarme(var_model(3, 1)[0], var_model(3, 1)[1])
HISTORY_4, PREDICTIONS_4 = forecast_to_datafarme(var_model(4, 1)[0], var_model(4, 1)[1])
HISTORY_5, PREDICTIONS_5 = forecast_to_datafarme(var_model(5, 1)[0], var_model(5, 1)[1])
HISTORY_6, PREDICTIONS_6 = forecast_to_datafarme(var_model(6, 1)[0], var_model(6, 1)[1])
HISTORY_7, PREDICTIONS_7 = forecast_to_datafarme(var_model(7, 1)[0], var_model(7, 1)[1])
HISTORY_8, PREDICTIONS_8 = forecast_to_datafarme(var_model(8, 1)[0], var_model(8, 1)[1])
HISTORY_9, PREDICTIONS_9 = forecast_to_datafarme(var_model(9, 1)[0], var_model(9, 1)[1])
HISTORY_10, PREDICTIONS_10 = forecast_to_datafarme(var_model(10, 1)[0], var_model(10, 1)[1])

In [ ]:
PREDICTIONS = [naive_predictions, PREDICTIONS_1, PREDICTIONS_2, PREDICTIONS_3, PREDICTIONS_4,
               PREDICTIONS_5, PREDICTIONS_6, PREDICTIONS_7, PREDICTIONS_8, 
               PREDICTIONS_9, PREDICTIONS_10]

LAGS = [i for i in range(1, 11)]
LAGS = ['naive_forecast'] + LAGS 